In [1]:
from rouge import Rouge, FilesRouge
import pandas as pd
import numpy as np
import datetime
import glob
import os
rouge = Rouge()

In [2]:
# get max csv
# get max file
list_of_files = glob.glob('Z:/final_data/scored_data/notes_scored*.csv') # * means all if need specific format then *.csv
ckpt_text = max(list_of_files, key=os.path.getctime)
print(ckpt_text)

Z:/final_data/scored_data\notes_scored_2018-10-13_11_45_12.csv


In [3]:
# read in actual notes
notes = pd.read_csv(ckpt_text,index_col=0,keep_default_na=False)
notes.columns = ['findings','impressions','ref_impressions']

In [4]:
notes.head()

,findings,impressions,ref_impressions
29,No significant cardiopulmonary abnormality. T...,"['tracheostomy', 'tube', 'tip', 'approximately...",No significant abnormality.
33,ABDOMEN: LUNG BASES: Bibasilar scarring/atele...,"['1', 'no', 'evidence', 'of', 'bowel', 'obstru...",1. Normal appendix visualized within a right i...
35,Tracheostomy tube in place. Small lung volumes...,"['no', 'acute', 'change']",No acute abnormalities.
38,Normal cardiac and mediastinal contours. Trach...,"['no', 'specific', 'evidence', 'of', 'infection']",No acute cardiopulmonary abnormality.
39,LUNGS AND PLEURA: Chronic appearing basilar sc...,"['no', 'evidence', 'of', 'metastases', 'or', '...",No acute abnormality. Lower lung zone scarring...


In [9]:
notes.loc[39].impressions

"['no', 'evidence', 'of', 'metastases', 'or', 'other', 'significant', 'abnormality']"

In [33]:
# replacement of string
notes.loc[39].impressions.replace("[","").replace("]","").replace("'","").replace(",","")

'no evidence of metastases or other significant abnormality'

In [43]:
# replace outputted strings 
impressions_edit = []
for row in notes.impressions:
    tmpstr = row.replace("[","").replace("]","").replace("'","").replace(",","")
    impressions_edit.append(tmpstr)

In [44]:
impressions_edit[0:5]

['tracheostomy tube tip approximately 5 cm above the carina tracheostomy tube tip approximately 5 cm above the carina no significant pulmonary or pleural abnormalities tracheostomy tube tip approximately 5 cm above the carina tracheostomy tube tip approximately 5 cm above the carina',
 '1 no evidence of bowel obstruction 2 no evidence of bowel obstruction 3 no evidence of bowel obstruction',
 'no acute change',
 'no specific evidence of infection',
 'no evidence of metastases or other significant abnormality']

In [46]:
notes['pred_impressions'] = impressions_edit

In [47]:
notes.head()

,findings,impressions,ref_impressions,pred_impressions
29,No significant cardiopulmonary abnormality. T...,"['tracheostomy', 'tube', 'tip', 'approximately...",No significant abnormality.,tracheostomy tube tip approximately 5 cm above...
33,ABDOMEN: LUNG BASES: Bibasilar scarring/atele...,"['1', 'no', 'evidence', 'of', 'bowel', 'obstru...",1. Normal appendix visualized within a right i...,1 no evidence of bowel obstruction 2 no eviden...
35,Tracheostomy tube in place. Small lung volumes...,"['no', 'acute', 'change']",No acute abnormalities.,no acute change
38,Normal cardiac and mediastinal contours. Trach...,"['no', 'specific', 'evidence', 'of', 'infection']",No acute cardiopulmonary abnormality.,no specific evidence of infection
39,LUNGS AND PLEURA: Chronic appearing basilar sc...,"['no', 'evidence', 'of', 'metastases', 'or', '...",No acute abnormality. Lower lung zone scarring...,no evidence of metastases or other significant...


In [10]:
# join notes to the test set
test_dta = pd.read_csv("Z:/final_data/cohorts_merged_test.csv",keep_default_na=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,69,325) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
# join in the columns 
# test_dta.join(notes)
scored_df = pd.merge(test_dta, notes, left_index=True, right_index=True,how="left")

In [49]:
scored_df.loc[29:35]

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,visit_status_2,...,elecsig,patient_raw,rawtext,encounter_id_diagnosed,date_diagnosed,ild_status,findings_y,impressions_y,ref_impressions,pred_impressions
29,2068939,62,M,Black/African-American,Not Hispanic or Latino,,9294296,E,Completed,,...,**DATE<[**2019-05-20**]> 8:52 AM **NAME<RRR SS...,9294296_386626,"XR PORT CHEST 1V, **DATE<[**2019-05-20**]> 8:2...",,,0,No significant cardiopulmonary abnormality. T...,"['tracheostomy', 'tube', 'tip', 'approximately...",No significant abnormality.,tracheostomy tube tip approximately 5 cm above...
30,2068939,62,M,Black/African-American,Not Hispanic or Latino,,9294296,E,Completed,,...,,,,,,0,NaN,NaN,NaN,NaN
31,2068939,62,M,Black/African-American,Not Hispanic or Latino,,9339397,O,Completed,,...,,,,,,0,NaN,NaN,NaN,NaN
32,2068939,62,M,Black/African-American,Not Hispanic or Latino,,9339397,O,Completed,,...,,,,,,0,NaN,NaN,NaN,NaN
33,2068939,62,M,Black/African-American,Not Hispanic or Latino,,9354621,E,Completed,,...,**DATE<[**2019-09-11**]> 2:07 PM **NAME<VVV WW...,9354621_216872,CT UPPER ABD AND PELVIS W **DATE<[**2019-09-11...,,,0,ABDOMEN: LUNG BASES: Bibasilar scarring/atele...,"['1', 'no', 'evidence', 'of', 'bowel', 'obstru...",1. Normal appendix visualized within a right i...,1 no evidence of bowel obstruction 2 no eviden...
34,2068939,62,M,Black/African-American,Not Hispanic or Latino,,9354621,E,Completed,,...,,,,,,0,NaN,NaN,NaN,NaN
35,2068939,62,M,Black/African-American,Not Hispanic or Latino,,9401429,E,Completed,,...,**DATE<[**2019-06-13**]> 8:47 AM **NAME<LLL M...,9401429_420168,"XR PORT CHEST 1V, **DATE<[**2019-06-13**]> 6:2...",,,0,Tracheostomy tube in place. Small lung volumes...,"['no', 'acute', 'change']",No acute abnormalities.,no acute change


In [53]:
test_tmp = scored_df.loc[29:29]
test_tmp

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,visit_status_2,...,elecsig,patient_raw,rawtext,encounter_id_diagnosed,date_diagnosed,ild_status,findings_y,impressions_y,ref_impressions,pred_impressions
29,2068939,62,M,Black/African-American,Not Hispanic or Latino,,9294296,E,Completed,,...,**DATE<[**2019-05-20**]> 8:52 AM **NAME<RRR SS...,9294296_386626,"XR PORT CHEST 1V, **DATE<[**2019-05-20**]> 8:2...",,,0,No significant cardiopulmonary abnormality. T...,"['tracheostomy', 'tube', 'tip', 'approximately...",No significant abnormality.,tracheostomy tube tip approximately 5 cm above...


#### define functions

In [54]:
def avg_rouge_score(df,column_string):
    avg_score_list = []
    avg_score_cols = []
    for col in df.columns:
        if column_string in col:
            print("[info] string found", col)
            avg_score_cols.append(col)
            avgscore = np.nanmean(df[col])
            avg_score_list.append(avgscore)
    tmpdf = pd.DataFrame({
        'avg_score_cols':avg_score_cols,
        'score':avg_score_list
    })
    return(tmpdf)

In [55]:
def score_data(df, predicted, actuals):
    printcounter = 0
    for index,row in df.iterrows():
#         print(printcounter)
        printcounter += 1
        if(printcounter % 100000)==0:
            print("[info] 100000 iterations")
#         print("[info] index #: ",index)
#         print(row[predicted])
#         print(row[actuals])
        if str(row[predicted])!="nan":
#             print("[info] this was not a null row. Logging index")
            indexlist.append(index)
            rougescore = rouge.get_scores(row[predicted],row[actuals])
            tmpdf = pd.DataFrame(rougescore)
#             print("[info] now appending to list rouge values")
            for row in tmpdf:
                scorelist.append(tmpdf[row].apply(pd.Series).values[0])
        else:
            continue

#### now run through

In [56]:
# get np array now of rouge scores
scorelist = []
indexlist = []
score_data(scored_df,'pred_impressions','ref_impressions')

[info] 100000 iterations
[info] 100000 iterations


In [57]:
print(scorelist[0:5])
print(indexlist[0:5])

[array([0.11111111, 0.06666667, 0.33333333]), array([0., 0., 0.]), array([0.06878307, 0.06666667, 0.33333333]), array([0.21818182, 0.75      , 0.12765957]), array([0.0338983 , 0.11111111, 0.02      ])]
[29, 33, 35, 38, 39]


In [59]:
# assign
rouged_df = pd.DataFrame(np.array(scorelist).reshape(int(len(scorelist)/3),9),columns=['rouge-1-f','rouge-1-p','rouge-1-r','rouge-2-f','rouge-2-p','rouge-2-r','rouge-l-f','rouge-l-p','rouge-l-r'],index=indexlist)
rouged_df.head(n=10)


,rouge-1-f,rouge-1-p,rouge-1-r,rouge-2-f,rouge-2-p,rouge-2-r,rouge-l-f,rouge-l-p,rouge-l-r
29,0.111111,0.066667,0.333333,0.000000,0.000000,0.000000,0.068783,0.066667,0.333333
33,0.218182,0.750000,0.127660,0.033898,0.111111,0.020000,0.130714,0.750000,0.127660
35,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.333333,0.333333,0.333333
38,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
39,0.416667,0.625000,0.312500,0.080000,0.142857,0.055556,0.347222,0.625000,0.312500
42,0.083333,0.066667,0.111111,0.000000,0.000000,0.000000,0.074561,0.066667,0.111111
43,0.074074,0.200000,0.045455,0.000000,0.000000,0.000000,0.047248,0.200000,0.045455
112,0.058824,0.038462,0.125000,0.000000,0.000000,0.000000,0.040911,0.038462,0.125000
113,0.343750,0.550000,0.250000,0.176471,0.300000,0.125000,0.275755,0.550000,0.250000
117,0.255319,0.705882,0.155844,0.084746,0.263158,0.050505,0.161700,0.705882,0.155844


In [60]:
scored_rouged_df = pd.merge(scored_df, rouged_df, left_index=True, right_index=True,how="left")

In [62]:
scored_rouged_df.head(n=30)

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,visit_status_2,...,pred_impressions,rouge-1-f,rouge-1-p,rouge-1-r,rouge-2-f,rouge-2-p,rouge-2-r,rouge-l-f,rouge-l-p,rouge-l-r
0,2068882,28,M,Black/African-American,Not Hispanic or Latino,,12061505,E,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2068882,28,M,Black/African-American,Not Hispanic or Latino,,12061505,E,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2068893,73,F,White,Not Hispanic or Latino,,11869194,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2068893,73,F,White,Not Hispanic or Latino,,12180666,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2068893,73,F,White,Not Hispanic or Latino,,12180666,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2068893,73,F,White,Not Hispanic or Latino,,12180666,O,Completed,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2068904,75,M,White,Not Hispanic or Latino,,10396237,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2068904,75,M,White,Not Hispanic or Latino,,10396237,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2068904,75,M,White,Not Hispanic or Latino,,10396237,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2068904,75,M,White,Not Hispanic or Latino,,10559195,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
scored_rouged_df.columns

Index(['patient_id', 'age_at_last_encounter', 'gender', 'race', 'ethnicity',
       'date_of_death', 'encounter_id', 'enc_eio', 'visit_status_1',
       'visit_status_2',
       ...
       'pred_impressions', 'rouge-1-f', 'rouge-1-p', 'rouge-1-r', 'rouge-2-f',
       'rouge-2-p', 'rouge-2-r', 'rouge-l-f', 'rouge-l-p', 'rouge-l-r'],
      dtype='object', length=354)

In [64]:
avg_score_df = avg_rouge_score(scored_rouged_df,"rouge")
print(avg_score_df)

[info] string found rouge-1-f
[info] string found rouge-1-p
[info] string found rouge-1-r
[info] string found rouge-2-f
[info] string found rouge-2-p
[info] string found rouge-2-r
[info] string found rouge-l-f
[info] string found rouge-l-p
[info] string found rouge-l-r
  avg_score_cols     score
0      rouge-1-f  0.248022
1      rouge-1-p  0.339725
2      rouge-1-r  0.245002
3      rouge-2-f  0.110030
4      rouge-2-p  0.143200
5      rouge-2-r  0.115488
6      rouge-l-f  0.203831
7      rouge-l-p  0.325105
8      rouge-l-r  0.233514


In [65]:
# filename = "/gpfs/data/ildproject-share/final_data/scored_data/avg_score_%s.csv" % (str(datetime.datetime.now()).split('.')[0].replace(' ','_').replace(':','_'))
filename = "Z:/final_data/scored_data/avg_score_%s.csv" % (str(datetime.datetime.now()).split('.')[0].replace(' ','_').replace(':','_'))

In [ ]:
scored_rouged_df.to_csv("Z:/final_data/scored_data/scored_rouged_df.csv")

In [ ]:
avg_score_df.to_csv(filename)